# Advanced Tutorial on MetaSynth

In this tutorial, we will be creating a `generative metadata format` (`gmf`) metadata file from a dataset using MetaSynth. We are going to walk through some of the advanced abilities of MetaSynth, such as handling dates, setting distributions and ensuring uniqueness in columns. This example workflow starts from a `.csv` file as input, but it easily adapted to other formats. 

You can run this notebook by checking out the MetaSynth repo and installing metasynth with `pip install metasynth`

In [1]:
# %pip install metasynth

In [2]:
# import required packages
import datetime as dt
import pandas as pd
from metasynth import MetaDataset
from utils import get_demonstration_fp

## Step 1: Transforming your data into a pandas DataFrame

The first step in creating the metadata is reading and converting your dataset to a pandas DataFrame. 

In [3]:
demonstration_fp = get_demonstration_fp()
df = pd.read_csv(demonstration_fp)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22.0,0,7.2500,NaN,S,1922-03-23,14:57:38,2022-08-13 08:42:37
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,0,71.2833,C85,C,1938-03-22,17:16:30,2022-07-24 06:45:36
2,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,NaN,S,1909-11-06,15:57:59,2022-07-21 02:32:58
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,0,53.1000,C123,S,1915-10-14,14:17:36,2022-07-19 13:22:18
4,5,"Allen, Mr. William Henry",male,35.0,0,8.0500,NaN,S,1929-04-18,13:23:39,2022-07-26 11:47:42


MetaSynth will automatically generate the metadata from this DataFrame object so it is important to __ensure the data types for all the variables are correct__. For example, in the dataset above we see that Age is a floating point number whereas it should be an integer (22 instead of 22.0). In addition, there are a few categorical variables (Sex, Parch, Embarked) which are loaded in as string data types.

In general, we support [pandas dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes). For our example dataset we can specify the `dtypes` and load the dataset as follows:

In [4]:
dtypes = {
    "Survived": "category",  # Categories should be assigned this type.
    "Name": "string",  # Strings should be assigned like this
    "Age": "Int64",  # Integer columns that have NA's in them should be explicitly nullable integers.
    "Sex": "category",
    "SibSp": "category",
    "Parch": "category",
    "Ticket": "string",
    "Cabin": "string",
    "Embarked": "category",
}
df = pd.read_csv(demonstration_fp, dtype=dtypes)
df.head()

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,"Braund, Mr. Owen Harris",male,22,0,7.2500,<NA>,S,1922-03-23,14:57:38,2022-08-13 08:42:37
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,0,71.2833,C85,C,1938-03-22,17:16:30,2022-07-24 06:45:36
2,3,"Heikkinen, Miss. Laina",female,26,0,7.9250,<NA>,S,1909-11-06,15:57:59,2022-07-21 02:32:58
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,0,53.1000,C123,S,1915-10-14,14:17:36,2022-07-19 13:22:18
4,5,"Allen, Mr. William Henry",male,35,0,8.0500,<NA>,S,1929-04-18,13:23:39,2022-07-26 11:47:42


### Dates, times, and datetimes

One exception to the above is dates, times, and datetimes. Here, we use the types from the built-in `datetime` package. Thus we have to manually transform the strings in the columns with date, time, and datetimes to their proper objects. Since the columns in our example dataset follow the standard ISO-format, we can convert them with the `fromisoformat` method. If they are written in a different format, check out the [datetime library documentation](https://docs.python.org/3/library/datetime.html) on how to convert the strings to datetime/time/date objects.

In [5]:
df["Birthday"] = [dt.date.fromisoformat(x) for x in df["Birthday"]]
df["Board time"] = [dt.time.fromisoformat(x) for x in df["Board time"]]
df["Married since"] = [dt.datetime.fromisoformat(x) for x in df["Married since"]]

Now, let's check the data types of our DataFrame:

In [6]:
df.dtypes

PassengerId               int64
Name                     string
Sex                    category
Age                       Int64
Parch                  category
Fare                    float64
Cabin                    string
Embarked               category
Birthday                 object
Board time               object
Married since    datetime64[ns]
dtype: object

We see that most variables are now nicely specified as strings, categories and ints where necessary. For the dates and times we just created, we see the dtype `object`. This is the "catch-all" dtype for pandas. But don't worry, these columns have the correct type and MetaSynth will deal with it correctly:

In [7]:
df["Birthday"][0]

datetime.date(1922, 3, 23)

## Step 2: Creating a MetaDataset object from a DataFrame

Now a lot of work has already gone into creating a properly formatted dataframe. This work pays off at this stage: let's convert the DataFrame to a meta_dataset structure with the default options. Note: this takes a little bit of time!

In [8]:
meta_dataset = MetaDataset.from_dataframe(df)

/home/qubix/Documents/shared_work/synthetic/meta-synth/metasynth/disttree.py:126: UserWarning: 
Variable PassengerId seems unique, but not set to be unique.
Set the variable to be either unique or not unique to remove this warning.

  warnings.warn(f"\nVariable {series.name} seems unique, but not set to be unique.\n"


Then, we can show the metadata as a dictionary:

In [9]:
print(meta_dataset)

# Rows: 891
# Columns: 11

{'name': 'PassengerId', 'description': None, 'type': 'discrete', 'dtype': 'int64', 'prop_missing': 0.0, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 1, 'high': 892}}"}

{'name': 'Name', 'description': None, 'type': 'string', 'dtype': 'string', 'prop_missing': 0.0, 'distribution': '.[]{12,82}'}

{'name': 'Sex', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameters': {'labels': array(['female', 'male'], dtype='<U6'), 'probs': array([0.35241302, 0.64758698])}}"}

{'name': 'Age', 'description': None, 'type': 'discrete', 'dtype': 'Int64', 'prop_missing': 0.19865319865319866, 'distribution': "{'name': 'DiscreteUniformDistribution', 'parameters': {'low': 0, 'high': 81}}"}

{'name': 'Parch', 'description': None, 'type': 'categorical', 'dtype': 'category', 'prop_missing': 0.0, 'distribution': "{'name': 'MultinoulliDistribution', 'parameter

## Step 3: Saving the metadata in a file

After creating the metadata, we can save it to a file. The default format is `json`, meaning the file is quite legible by humans and computers alike. Therefore, it can be checked by the data controller and, when the disclosure risk is deemed to be low, this file can be shared with others.

In [10]:
file_path = "demonstration_metadata.json"
meta_dataset.to_json(file_path)

## Step 4: Generating synthetic data from the metadata

Upon receiving this file, you can use the MetaSynth package to generate a synthetic version of the dataset:

In [11]:
new_meta_dataset = MetaDataset.from_json(file_path)
new_meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,499,"k& [f~>REH;+n`(m7RauB20JW(QnEJ:iCg""268L2q_4.@...",female,75,2,55.194127,<NA>,S,1938-01-15,13:47:05,2022-08-07 10:23:29
1,432,giFY#>=?}1>oSWl})?X4.oU&p9nbt<PRM^)? 56XW>,male,31,0,7.331161,<NA>,S,1919-10-16,12:45:02,2022-08-14 10:05:52
2,777,"Fc1&P'Kc# J)3+^)&QyQ((;Tt*g@|yPyu@VOj,oPrJ|R0 ...",male,20,2,1.265007,<NA>,S,1927-06-22,13:53:52,2022-08-06 22:47:13
3,258,%rdZo7<Zb'(N0-8dsK:r/EBg)bl^E9$+Sn9lP'> lsEgkh...,male,23,0,7.986453,<NA>,S,1919-03-05,17:23:32,2022-08-02 23:14:25
4,692,rAf.7tO:a; m</Y2hVq A8$,male,44,0,142.386305,<NA>,S,1928-07-06,13:23:33,2022-08-04 00:56:32


As you can see, the fake data looks a lot like the real data! However, it could still use some improvement. In the next sections, we will explore manual changes we can make to improve the quality of the synthetic data.

## Step 5: Improving the quality of the synthetic data

### Set unique columns

One column (PassengerId) has been detected as possibly unique by MetaSynth, as indicated by the following warning:

> "Variable PassengerId seems unique, but not set to be unique."

This column holds a variable with unique passenger identifiers, so in fact we do want synthetic data generated for this column to be unique as well. We can add this to the metadata by creating a list of options which we call a `specification`, or `spec`:

In [12]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}
}

# then, we add that dictionary as the `spec` argument
meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)

# then, let's check what the metadata about PassengerId contains!
meta_dataset["PassengerId"].to_dict()

{'name': 'PassengerId',
 'type': 'discrete',
 'dtype': 'int64',
 'prop_missing': 0.0,
 'distribution': {'name': 'UniqueKeyDistribution',
  'parameters': {'low': 1, 'consecutive': 1}}}

So let's check what is generated from this new metadata:

In [13]:
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,H hzXfU(j.=NZ>3qNB mGaz2&YQiM F|(&g~vb]--c4V;=...,male,50,0,57.530086,<NA>,S,1915-07-22,14:13:13,2022-07-17 08:18:29
1,2,"#v/1[JFi`%lbhMHo]$Lz}exAle8B<enBHu""xn?C4%%)=t'...",female,37,0,35.268956,<NA>,S,1904-02-09,12:28:25,2022-08-14 20:56:22
2,3,7wUJ7cf 3\Dk[Wc[/BQf)it3o#RJ`$J}>LeE \,male,71,1,39.696053,<NA>,C,1916-08-14,18:37:52,2022-07-27 13:55:33
3,4,V.oX^s|7/`&Q_5Z o+5`:>{~YEI}j#?cA/RKGt?Y'GV c+$M,female,30,2,26.521319,<NA>,S,1921-05-30,16:50:19,2022-07-29 15:43:43
4,5,"mRpaYt~\Amgpc^')j8&HD Q%@/X4iq}sH!fNQdc1(,TL:X...",male,73,0,30.068923,A(qXsW% f#8]5 -,S,1927-02-21,13:04:09,2022-07-19 21:31:09


### Fake names (and others)

As one can see, the `Name` of the passengers is not quite so well synthesized. The reason is that the string type interpreter in MetaSynth is designed for `structured` strings (like room numbers such as `B1.09`, `B1.01` or `A1.08`) and not unstructured strings. However, MetaSynth supports the [faker](https://faker.readthedocs.io/en/master/index.html) package, which includes a lot of data types that it can fake. The columns using faker are not based on the real data at all so they do not disclose any info about the real data.

We fake names as follows:

In [14]:
# First, we create a specification dictionary for the variables
var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Keith Gould,male,33,0,2.664089,BDR#Or.``-)M,Q,1907-01-15,15:42:21,2022-08-10 13:56:55
1,2,Albert Williams,male,<NA>,0,24.586148,<NA>,S,1932-10-13,18:20:17,2022-07-31 10:30:49
2,3,Kevin Martinez,female,28,0,127.454940,<NA>,S,1904-02-11,18:16:20,2022-07-22 06:27:53
3,4,Danielle Blake,male,74,0,57.929173,<NA>,S,1923-03-14,18:30:22,2022-08-07 18:12:39
4,5,Natalie Davis,male,1,0,9.619851,<NA>,S,1924-02-14,11:51:40,2022-07-23 02:52:14


### Set distributions manually

Without user input, the distribution chosen for each variable is inferred by choosing the best fitting from available distributions for the variable type. However, we can also manually specify which distribution to fit, or we can even just fully specify how the variable should be generated.

In [15]:
from metasynth.distribution import DiscreteUniformDistribution

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)} # fully specify a distribution for age (uniform between 20 and 40)
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(5)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Wayne Smith,male,38,0,2.856833,<NA>,S,1914-12-14,17:57:32,2022-08-12 19:11:03
1,2,Crystal Martinez,male,21,0,2.582094,<NA>,S,1912-08-01,15:13:41,2022-08-03 18:57:32
2,3,Alexis Jones,male,33,0,3.576966,<NA>,S,1939-12-26,18:33:31,2022-07-26 18:49:12
3,4,James Hicks,male,22,0,1.284583,<NA>,S,1910-05-03,11:15:57,2022-08-05 01:41:32
4,5,Michelle Terry MD,male,31,0,4.285548,<NA>,S,1930-11-30,11:12:31,2022-07-21 12:08:37


### Specifying the distribution of structured strings

For more or less structured strings, we can manually set the structure of the strings based on regular expressions. For example, we see that most Cabins are structured like [A-F] and then 2 or 3 digit numbers. We can include this as follows:

In [16]:
from metasynth.distribution import RegexDistribution

# To create a regex distribution, you need a list of tuples, where each tuple is an element.
# The first part of the tuple is a string representation of the regex, while the second is the proportion of the
# time the regex element is used.
cabin_distribution = RegexDistribution(r"[ABCDEF]\d{2,3}")  # Add the r so that it becomes a literal string.
# just for completeness: data generated from this distribution will always match the regex [ABCDEF]?(\d{2,3})?

var_spec = {
    "PassengerId": {"unique": True}, 
    "Name": {"distribution": "faker.name"},
    "Fare": {"distribution": "LogNormalDistribution"}, # estimate / fit a log-normal distribution based on the data
    "Age": {"distribution": DiscreteUniformDistribution(20, 40)}, # fully specify a distribution for age (uniform between 20 and 40)
    "Cabin": {"distribution": cabin_distribution}
}

meta_dataset = MetaDataset.from_dataframe(df, spec=var_spec)
meta_dataset.synthesize(10)

,PassengerId,Name,Sex,Age,Parch,Fare,Cabin,Embarked,Birthday,Board time,Married since
0,1,Matthew Clayton,female,25,0,0.265496,<NA>,C,1911-08-08,12:42:28,2022-08-11 23:12:39
1,2,Austin Brooks,male,35,1,0.482256,<NA>,S,1907-09-02,15:14:37,2022-07-30 16:45:02
2,3,Jessica Carter,male,35,0,4.206140,<NA>,S,1904-10-05,15:09:41,2022-07-26 10:06:47
3,4,Timothy Martinez,female,<NA>,0,0.345499,<NA>,S,1931-08-19,16:09:37,2022-07-22 10:32:19
4,5,Mark Merritt,female,20,0,0.341555,A478,S,1904-08-03,16:09:16,2022-07-21 01:58:47
5,6,Christopher Rogers,male,33,0,0.429491,<NA>,Q,1921-12-12,17:30:13,2022-07-29 13:54:14
6,7,Dr. Cynthia Solis DDS,female,22,0,2.619001,F763,S,1921-09-06,11:15:10,2022-07-19 12:20:05
7,8,Kenneth Becker MD,female,27,0,0.094648,<NA>,S,1933-12-29,17:01:18,2022-07-16 06:00:22
8,9,Elizabeth Thompson,male,25,0,1.437711,<NA>,C,1923-01-06,11:46:56,2022-07-25 06:18:15
9,10,Julie Brown,male,23,2,0.596881,<NA>,S,1921-10-12,11:59:21,2022-07-22 05:48:42
